### TODO
* Explain: Approximations
* Explain: Linear approx.
* Explain: Q-learning extension
* Explain: SARSA extension
* Explain: Mountain Car env
* code: Run and compare (on mountain car)


### DONE


### NOTES

# Linear Approximate Methods for Reinforcement Learning


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
